In [102]:
# Set covering problem: you need to select a certain number of blocks in order to cover all the initial sets
# Example: find the minimum number of items that together guarantee to cover something
# Definitions

# STATE = contains all the relevant informations to represent the current situation

In [103]:
from random import random
import numpy as np
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

In [104]:
PROBLEM_SIZE = 15
NUM_SETS = 20

In [105]:
sets = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

"""
PROBLEM_SIZE = 3
NUM_SETS = 8
sets = ([False, False, False],
       [False, False, True],
       [False, False, False],
       [True, False, False],
       [True, False, True],
       [False, False, True],
       [False, True, False],
       [False, True, True])
"""

'\nPROBLEM_SIZE = 3\nNUM_SETS = 8\nsets = ([False, False, False],\n       [False, False, True],\n       [False, False, False],\n       [True, False, False],\n       [True, False, True],\n       [False, False, True],\n       [False, True, False],\n       [False, True, True])\n'

In [106]:
def goal_check(state):
  #np.all = checks whether all elements are TRUE
  #reduce = applies the logical or to all elements of the specified iterable
  return np.all(reduce(np.logical_or, [sets[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [107]:
assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\Lorenzo\AppData\Local\Temp/ipykernel_11804/987543073.py:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")


In [108]:
from collections import namedtuple
State = namedtuple('State', ['taken', 'not_taken'])

In [109]:
def count_covered(state):
    return reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def h6(state):
    already_covered = count_covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    not_covered = np.logical_not(already_covered)
    candidates = sorted((sum(np.logical_and(s, not_covered)) for s in sets), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken


In [113]:
# A* algorithm implementation

def is_problem_solvable(sets):
    # this function checks that each individual point is covered by at least one set
    row = False
    for j in range(0, PROBLEM_SIZE):
        for i in range(0, NUM_SETS):
            row |= sets[i][j]
        if row == False:
            return False
        else:
            row = False
    return True

def goal_check(state):
    # This is a binary output: did we reach a solution? True of False
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def h(state):
    # optimistic heuristic by PROF: let's count the points we are missing to cover over
    # the largest set (where largest means the set with the greatest amount of trues) 
    missing_points = PROBLEM_SIZE - sum(count_covered(state))
    largest_set = np.max(np.array(sets).sum(axis=1))
    # That is: how many sets are needed (optimistically) to cover the universe?
    return ceil(missing_points/largest_set)

def h2(state):
    missing_points = PROBLEM_SIZE - sum(count_covered(state))
    mean_setsize_nt = np.mean([sum(sets[i]) for i in state.not_taken]) #mediamente quanti true ci sono in un set
    #largest_set = np.max(np.array(sets).sum(axis=1))
    return ceil(missing_points/mean_setsize_nt)

#This is my heuristic!
def h4(current_state):
    return -sum(count_covered(current_state))

def g(state):
    return len(state.taken)

def f(state):
    return g(state) + h6(state)

def print_sets():
  print("PRINTING SETS:")
  for s in range(0,NUM_SETS):
    print(f"{s}) {sets[s]}")
  print("-"*25)

print_sets()
if(not is_problem_solvable(sets)):
  print("[!] This configuration of the problem is not solvable! Some point is not covered by any set.")
else: 
    frontier = PriorityQueue()
    state = State(set(), set(range(NUM_SETS)))
    frontier.put((f(state), state))

    counter = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            print(f"{counter}) Putting f()={f(new_state)}, g()={g(new_state)}, h()={f(new_state)-len(new_state.taken)} with action {new_state}")
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
    print(
        f"Solved in {counter:,} steps ({len(current_state.taken)} tiles. Sets used to cover {current_state.taken})"
    )

PRINTING SETS:
0) [False False  True False False False False False False False False False
 False  True False]
1) [False False False False  True False False False False False False False
 False False False]
2) [False False False  True False False False False False False False False
 False False  True]
3) [False False False False False False False  True False False  True False
 False  True False]
4) [ True False False  True False  True False False False False  True False
 False False False]
5) [False False False False False False False  True False False False False
 False  True  True]
6) [False False  True False False False  True False False False False False
 False False False]
7) [ True False False  True False False False False False False  True False
 False  True False]
8) [ True  True False False False False False  True False False  True False
  True False False]
9) [False  True False  True False False  True False False  True False False
 False False False]
10) [False False False Fa

4) Putting f()=6, g()=2, h()=4 with action State(taken={3, 6}, not_taken={0, 1, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h()=3 with action State(taken={4, 6}, not_taken={0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h()=3 with action State(taken={5, 6}, not_taken={0, 1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h()=3 with action State(taken={6, 7}, not_taken={0, 1, 2, 3, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=6, g()=2, h()=4 with action State(taken={8, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h()=3 with action State(taken={9, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h()=3 with action State(taken={10, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19})
4) Putting f()=5, g()=2, h(

8) Putting f()=5, g()=2, h()=3 with action State(taken={18, 12}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19})
8) Putting f()=5, g()=2, h()=3 with action State(taken={19, 12}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18})
9) Putting f()=5, g()=2, h()=3 with action State(taken={0, 11}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
9) Putting f()=5, g()=2, h()=3 with action State(taken={1, 11}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
9) Putting f()=5, g()=2, h()=3 with action State(taken={2, 11}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
9) Putting f()=5, g()=2, h()=3 with action State(taken={11, 3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
9) Putting f()=5, g()=2, h()=3 with action State(taken={11, 4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
9) Putting f()=5, g()=2, h(

12) Putting f()=5, g()=2, h()=3 with action State(taken={8, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
12) Putting f()=6, g()=2, h()=4 with action State(taken={9, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
12) Putting f()=5, g()=2, h()=3 with action State(taken={10, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19})
12) Putting f()=5, g()=2, h()=3 with action State(taken={11, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19})
12) Putting f()=6, g()=2, h()=4 with action State(taken={12, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19})
12) Putting f()=6, g()=2, h()=4 with action State(taken={13, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19})
12) Putting f()=5, g()=2, h()=3 with action State(taken={14, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19})
12) Putting f()=5, g

16) Putting f()=6, g()=2, h()=4 with action State(taken={9, 3}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g()=2, h()=3 with action State(taken={9, 4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g()=2, h()=3 with action State(taken={9, 5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g()=2, h()=3 with action State(taken={9, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=6, g()=2, h()=4 with action State(taken={9, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g()=2, h()=3 with action State(taken={8, 9}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g()=2, h()=3 with action State(taken={9, 10}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19})
16) Putting f()=5, g

...

932) Putting f()=7, g()=4, h()=3 with action State(taken={16, 19, 13, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 15, 17, 18})
932) Putting f()=6, g()=4, h()=2 with action State(taken={16, 19, 14, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 17, 18})
932) Putting f()=6, g()=4, h()=2 with action State(taken={16, 19, 6, 15}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18})
932) Putting f()=6, g()=4, h()=2 with action State(taken={16, 17, 19, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18})
932) Putting f()=6, g()=4, h()=2 with action State(taken={16, 18, 19, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17})
933) Putting f()=7, g()=4, h()=3 with action State(taken={0, 18, 19, 7}, not_taken={1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17})
933) Putting f()=6, g()=4, h()=2 with action State(taken={1, 18, 19, 7}, not_taken={0, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17})
933) Putting 

936) Putting f()=7, g()=4, h()=3 with action State(taken={16, 17, 19, 4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 18})
936) Putting f()=7, g()=4, h()=3 with action State(taken={16, 18, 19, 4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17})
937) Putting f()=7, g()=4, h()=3 with action State(taken={0, 16, 3, 19}, not_taken={1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18})
937) Putting f()=7, g()=4, h()=3 with action State(taken={16, 1, 3, 19}, not_taken={0, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18})
937) Putting f()=7, g()=4, h()=3 with action State(taken={16, 19, 2, 3}, not_taken={0, 1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18})
937) Putting f()=7, g()=4, h()=3 with action State(taken={16, 19, 3, 4}, not_taken={0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18})
937) Putting f()=7, g()=4, h()=3 with action State(taken={16, 19, 3, 5}, not_taken={0, 1, 2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18})
937) Putting 

941) Putting f()=7, g()=6, h()=1 with action State(taken={1, 8, 11, 17, 18, 19}, not_taken={0, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16})
942) Putting f()=7, g()=5, h()=2 with action State(taken={0, 11, 16, 18, 19}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting f()=6, g()=5, h()=1 with action State(taken={1, 11, 16, 18, 19}, not_taken={0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting f()=7, g()=5, h()=2 with action State(taken={2, 11, 16, 18, 19}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting f()=7, g()=5, h()=2 with action State(taken={3, 11, 16, 18, 19}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting f()=7, g()=5, h()=2 with action State(taken={4, 11, 16, 18, 19}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting f()=7, g()=5, h()=2 with action State(taken={5, 11, 16, 18, 19}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15, 17})
942) Putting 

946) Putting f()=7, g()=4, h()=3 with action State(taken={11, 3, 12, 13}, not_taken={0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=7, g()=4, h()=3 with action State(taken={13, 11, 4, 12}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=6, g()=4, h()=2 with action State(taken={13, 11, 12, 5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=7, g()=4, h()=3 with action State(taken={11, 12, 13, 6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=8, g()=4, h()=4 with action State(taken={11, 12, 13, 7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=7, g()=4, h()=3 with action State(taken={8, 11, 12, 13}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 14, 15, 16, 17, 18, 19})
946) Putting f()=7, g()=4, h()=3 with action State(taken={9, 11, 12, 13}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 14, 15, 16, 17, 18, 19})
946) Putting 

950) Putting f()=7, g()=5, h()=2 with action State(taken={6, 11, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15, 16})
950) Putting f()=7, g()=5, h()=2 with action State(taken={7, 11, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16})
950) Putting f()=6, g()=5, h()=1 with action State(taken={8, 11, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16})
950) Putting f()=7, g()=5, h()=2 with action State(taken={9, 11, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16})
950) Putting f()=7, g()=5, h()=2 with action State(taken={10, 11, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16})
950) Putting f()=7, g()=5, h()=2 with action State(taken={11, 12, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16})
950) Putting f()=7, g()=5, h()=2 with action State(taken={11, 13, 17, 18, 19}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16})
950) Putting 